# Executive Pipeline - Sunum Notebook'u

Bu notebook, yöneticilere sunum yaparken kullanılmak üzere tasarlandı. Parametreler değiştirilerek farklı senaryolar hızlıca test edilebilir.

**Kullanım:** Sunum sırasında bir soru geldiğinde, ilgili hücredeki parametreyi değiştirip tekrar çalıştırın.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
project_root = Path("..").resolve()
sys.path.append(str(project_root))

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine, text
from src.config import DATABASE_URL

engine = create_engine(DATABASE_URL)
print("Bağlantı hazır")

## 1. Genel Bakış

Temel iş metrikleri.

In [ ]:
query = """
SELECT 
    COUNT(DISTINCT o.order_id) as siparis,
    COUNT(DISTINCT c.customer_unique_id) as musteri,
    SUM(oi.price) as gelir,
    AVG(r.review_score) as puan
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
LEFT JOIN order_reviews r ON o.order_id = r.order_id
WHERE o.order_status = 'delivered'
"""
m = pd.read_sql(text(query), engine).iloc[0]

print(f"Sipariş: {m['siparis']:,.0f}")
print(f"Müşteri: {m['musteri']:,.0f}")
print(f"Gelir: R$ {m['gelir']:,.2f}")
print(f"Puan: {m['puan']:.2f}/5")

## 2. Teslimat Analizi

Eyaletlere göre teslimat süresi. Model RMSE: 7.60 gün.

In [ ]:
query = """
SELECT 
    EXTRACT(EPOCH FROM (order_delivered_customer_date::timestamp - order_purchase_timestamp::timestamp))/86400 as gun,
    c.customer_state as eyalet
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL
LIMIT 50000
"""
df = pd.read_sql(text(query), engine)

# Eyalet bazlı ortalama
eyalet = df.groupby('eyalet')['gun'].agg(['mean', 'count']).reset_index()
eyalet.columns = ['Eyalet', 'Ort', 'N']
eyalet = eyalet[eyalet['N'] > 500].sort_values('Ort', ascending=False)

fig = px.bar(eyalet, x='Eyalet', y='Ort', color='Ort', color_continuous_scale='RdYlGn_r',
             title='Eyalet Bazlı Teslimat Süresi (gün)')
fig.show()

## 3. Müşteri Segmentleri

RFM bazlı 5 segment. Her segment için farklı strateji uygulanabilir.

In [ ]:
query = 'SELECT "Cluster", COUNT(*) as n, AVG("Monetary") as harcama FROM customer_segments GROUP BY "Cluster"'
seg = pd.read_sql(query, engine)

isimler = {0: 'Sadık', 1: 'Potansiyel', 2: 'Riskli', 3: 'Şampiyon', 4: 'Uyuyan'}
seg['Segment'] = seg['Cluster'].map(isimler)

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'pie'}, {'type':'bar'}]])
fig.add_trace(go.Pie(labels=seg['Segment'], values=seg['n']), row=1, col=1)
fig.add_trace(go.Bar(x=seg['Segment'], y=seg['harcama'], marker_color='#2ecc71'), row=1, col=2)
fig.update_layout(title='Segment Dağılımı ve Ortalama Harcama', showlegend=False)
fig.show()

## 4. Kampanya Simülasyonu

**Parametreleri değiştirerek farklı senaryolar deneyin:**

In [ ]:
# === PARAMETRELERİ DEĞİŞTİRİN ===
INDIRIM = 0.10        # %10 indirim
HEDEF_SEGMENT = 2     # Riskli müşteriler
ULASIM = 0.30         # %30 müşteriye ulaşıyoruz
# ================================

query = f'SELECT COUNT(*) as n, AVG("Monetary") as m FROM customer_segments WHERE "Cluster" = {HEDEF_SEGMENT}'
data = pd.read_sql(query, engine).iloc[0]

toplam = int(data['n'])
harcama = data['m']
ulasan = int(toplam * ULASIM)
kazanilan = ulasan * 0.20  # %20 geri kazanım varsayımı
gelir = kazanilan * harcama
maliyet = ulasan * harcama * INDIRIM * 0.5
kar = gelir - maliyet

print(f"Hedef: {isimler.get(HEDEF_SEGMENT)} ({toplam:,} kişi)")
print(f"Ulaşılan: {ulasan:,}")
print(f"Kazanılan: {kazanilan:,.0f}")
print(f"Net kar: R$ {kar:,.2f}")
print(f"ROI: %{(kar/maliyet)*100:.0f}")

## 5. Aylık Trend

In [ ]:
query = """
SELECT DATE_TRUNC('month', order_purchase_timestamp::timestamp) as ay,
       COUNT(*) as siparis, SUM(oi.price) as gelir
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY DATE_TRUNC('month', order_purchase_timestamp::timestamp)
ORDER BY ay
"""
trend = pd.read_sql(text(query), engine)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=trend['ay'], y=trend['siparis'], name='Sipariş', opacity=0.7), secondary_y=False)
fig.add_trace(go.Scatter(x=trend['ay'], y=trend['gelir'], name='Gelir', mode='lines+markers'), secondary_y=True)
fig.update_layout(title='Aylık Sipariş ve Gelir')
fig.show()

## Model Özeti

| Model | Metrik | Değer |
|-------|--------|-------|
| Teslimat | RMSE | 7.60 gün |
| Churn | Rate | %80.3 |
| Öneri | Coverage | 99K |

**Kullanılan özellikler:** mesafe (403 km ort), satıcı puanı (4.04 ort), kargo, fiyat, ağırlık